In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mkl-fft to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-random to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-umath to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 108.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install "numpy<2" --force-reinstall
!pip install --upgrade matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 83.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.4.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
google-colab 1.0.0 requires notebook==6.5.7, but you have notebook 6.5.4 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
google-colab 1.0.0 requires requests==2.32

In [20]:
import os, shutil, random, glob

SRC = "/kaggle/input/cocotext-v20/data"            # where all .jpg/.txt currently are
OUT = "/kaggle/working/cocotext-yolo"              # output root
split_ratio = 0.90

# Create output folders
for split in ("train","val"):
    os.makedirs(os.path.join(OUT,"images",split), exist_ok=True)
    os.makedirs(os.path.join(OUT,"labels",split), exist_ok=True)

# Find images (recursive), use only common image extensions
img_paths = []
for ext in ("*.jpg","*.jpeg","*.png","*.bmp","*.webp"):
    img_paths.extend(glob.glob(os.path.join(SRC, ext)))
img_paths = sorted(img_paths)
random.shuffle(img_paths)

print("Found images:", len(img_paths))
if not img_paths:
    raise SystemExit("No images found in SRC. Check SRC path.")

# Split
train_count = int(len(img_paths) * split_ratio)
train_imgs = img_paths[:train_count]
val_imgs = img_paths[train_count:]

def copy_list(img_list, split):
    missing = 0
    for img_full in img_list:
        base = os.path.splitext(os.path.basename(img_full))[0]
        label_src = os.path.join(SRC, base + ".txt")
        img_dst = os.path.join(OUT, "images", split, base + os.path.splitext(img_full)[1].lower())
        label_dst = os.path.join(OUT, "labels", split, base + ".txt")
        shutil.copy(img_full, img_dst)
        if os.path.exists(label_src):
            shutil.copy(label_src, label_dst)
        else:
            missing += 1
            if missing <= 10:
                print("Missing label:", label_src)

    print(f"Copied {len(img_list)} items to {split} (missing labels: {missing})")

copy_list(train_imgs, "train")
copy_list(val_imgs, "val")

# Quick verification counts
import glob as _glob
print("Train images:", len(_glob.glob(os.path.join(OUT,"images","train","*"))))
print("Train labels:", len(_glob.glob(os.path.join(OUT,"labels","train","*"))))
print("Val   images:", len(_glob.glob(os.path.join(OUT,"images","val","*"))))
print("Val   labels:", len(_glob.glob(os.path.join(OUT,"labels","val","*"))))

print("Done.")


Found images: 17141
Copied 15426 items to train (missing labels: 0)
Copied 1715 items to val (missing labels: 0)
Train images: 15426
Train labels: 15426
Val   images: 1715
Val   labels: 1715
Done.


In [21]:
yaml_content = """
path: /kaggle/working/cocotext-yolo

train: /kaggle/working/cocotext-yolo/images/train
val: /kaggle/working/cocotext-yolo/images/val

names:
  0: text
"""

with open("/kaggle/working/cocotext-yolo/cocotext.yaml", "w") as f:
    f.write(yaml_content)

In [11]:
import torch
torch.cuda.is_available()


True

In [ ]:
from ultralytics import YOLO

model = YOLO("yolo11s.pt")

model.train(
    data="/kaggle/working/cocotext-yolo/cocotext.yaml",
    epochs=20,
    imgsz=640,
    batch=32,
    device='0,1'  
)

Ultralytics 8.3.233 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
                                                       CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/cocotext-yolo/cocotext.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train5, nbs=64, nms=False, opset=None, op